In [77]:
#Todos os imports necessários para aplicação.
import os
import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import classpath_jars
from pyspark.sql.functions import create_map, struct
from pyspark.sql.functions import exp
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import avg
from pyspark.sql.functions import col
from pyspark.sql.functions import when, lit
from pyspark.sql import SQLContext
from functools import reduce 
from pyspark.sql import DataFrame

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [13]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("Dados Enem")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark

In [14]:
LANDED = 'C:\\BigData\\Fontes\\microdados_enem\\Landed\\'
RAW = 'C:\\BigData\\Fontes\\microdados_enem\\Raw\\'
MODELED = 'C:\\BigData\\Fontes\\microdados_enem\\Modeled\\'
SELF = 'C:\\BigData\\Fontes\\microdados_enem\\Self\\'

In [15]:
ENEM_PARQUET = 'MICRODADOS_ENEM_UNION.parquet'

In [16]:
sqlContext = SQLContext(spark.sparkContext)

df = spark.read.parquet(MODELED + ENEM_PARQUET)    
df.printSchema()  

root
 |-- NU_ANO: integer (nullable = true)
 |-- NU_NOTA_CN: double (nullable = true)
 |-- NU_NOTA_CH: double (nullable = true)
 |-- NU_NOTA_LC: double (nullable = true)
 |-- NU_NOTA_MT: double (nullable = true)
 |-- NU_NOTA_REDACAO: integer (nullable = true)
 |-- Q001: string (nullable = true)
 |-- Q002: string (nullable = true)
 |-- Q005: integer (nullable = true)
 |-- RENDA_FAMILIAR: string (nullable = true)



In [84]:
dfPorRendaFamiliar = df.selectExpr("NU_ANO", "RENDA_FAMILIAR",  "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO"  ).groupBy("NU_ANO", "RENDA_FAMILIAR").avg("NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO").orderBy("NU_ANO", "RENDA_FAMILIAR")
dfPorRendaFamiliar.printSchema() 

root
 |-- NU_ANO: integer (nullable = true)
 |-- RENDA_FAMILIAR: string (nullable = true)
 |-- avg(NU_NOTA_CN): double (nullable = true)
 |-- avg(NU_NOTA_CH): double (nullable = true)
 |-- avg(NU_NOTA_LC): double (nullable = true)
 |-- avg(NU_NOTA_MT): double (nullable = true)
 |-- avg(NU_NOTA_REDACAO): double (nullable = true)



DataFrame[NU_ANO: int, RENDA_FAMILIAR: string, avg(NU_NOTA_CN): double, avg(NU_NOTA_CH): double, avg(NU_NOTA_LC): double, avg(NU_NOTA_MT): double, avg(NU_NOTA_REDACAO): double]

In [85]:
dfPorRendaFamiliar = dfPorRendaFamiliar.withColumn("NOTA_GERAL", ( (col("avg(NU_NOTA_LC)") + col("avg(NU_NOTA_MT)") + col("avg(NU_NOTA_CN)") + col("avg(NU_NOTA_CH)")) / 4  + col("avg(NU_NOTA_REDACAO)" ) ) / lit(2) ) 

In [122]:
dfPorRendaFamiliarFinal = dfPorRendaFamiliar.select("NU_ANO", "RENDA_FAMILIAR", "NOTA_GERAL")


dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("A") , lit("Nenhuma Renda") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("B") , lit("Até R$ 880,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("C") , lit("De R$ 880,01 até R$ 1.320,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("D") , lit("De R$ 1.320,01 até R$ 1.760,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("E") , lit("De R$ 1.760,01 até R$ 2.200,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("F") , lit("De R$ 2.200,01 até R$ 2.640,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("G") , lit("De R$ 2.640,01 até R$ 3.520,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("H") , lit("De R$ 3.520,01 até R$ 4.400,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("I") , lit("De R$ 4.400,01 até R$ 5.280,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("J") , lit("De R$ 5.280,01 até R$ 6.160,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("K") , lit("De R$ 6.160,01 até R$ 7.040,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("L") , lit("De R$ 7.040,01 até R$ 7.920,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("M") , lit("De R$ 7.920,01 até R$ 8.800,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )


In [120]:
#dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("N") , lit("De R$ 8.800,01 até R$ 10.560,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
#dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("O") , lit("De R$ 10.560,01 até R$ 13.200,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
#dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("P") , lit("De R$ 13.200,01 até R$ 17.600,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )
#dfPorRendaFamiliarFinal = dfPorRendaFamiliarFinal.withColumn("RENDA_FAMILIAR", when(dfPorRendaFamiliarFinal.RENDA_FAMILIAR == lit("Q") , lit("Mais de R$ 17.600,00") ).otherwise(dfPorRendaFamiliarFinal.RENDA_FAMILIAR)  )


In [123]:
dfPorRendaFamiliarFinal.show()

+------+--------------------+------------------+
|NU_ANO|      RENDA_FAMILIAR|        NOTA_GERAL|
+------+--------------------+------------------+
|  2016|                null| 615.2583333333333|
|  2016|       Nenhuma Renda| 473.1718047992241|
|  2016|       Até R$ 880,00| 479.6661556176029|
|  2016|De R$ 880,01 até ...| 500.2549036511241|
|  2016|De R$ 1.320,01 at...| 515.0369947863906|
|  2016|De R$ 1.760,01 at...| 526.7168036240898|
|  2016|De R$ 2.200,01 at...| 537.7037669698568|
|  2016|De R$ 2.640,01 at...|  553.840504106677|
|  2016|De R$ 3.520,01 at...| 568.7492590274885|
|  2016|De R$ 4.400,01 at...| 584.5491117347406|
|  2016|De R$ 5.280,01 at...| 597.4382883025535|
|  2016|De R$ 6.160,01 at...| 606.8770430953273|
|  2016|De R$ 7.040,01 at...|  615.929418193052|
|  2016|De R$ 7.920,01 at...| 617.8437877296365|
|  2016|                   N| 624.9513490055512|
|  2016|                   O| 634.7675900554352|
|  2016|                   P| 644.3501952336267|
|  2016|            